# Connect google drive and check if GPU is available or not 

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

# Mount to google drive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'  'Getting started.pdf'   yolov3


# Clone the Darknet into colab



In [4]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15363, done.
remote: Total 15363 (delta 0), reused 0 (delta 0), pack-reused 15363
Receiving objects: 100% (15363/15363), 13.98 MiB | 17.31 MiB/s, done.
Resolving deltas: 100% (10332/10332), done.


# Compile Darknet using colab GPU


In [ ]:
# Changing makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

[Errno 20] Not a directory: 'darknet'
/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
     

# Configure Darknet network for training YOLO V3 based on your dataset and class types

In [6]:
# Making a seperate copy of standard yolo config file for creating custom training config file 
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [7]:
# Since we are working with images of 5 classes (Person, Bus, Car, Bicycle and TrafficLight) for this project
# No of classes will be 5

# Making changes with model configurations accordingly inside the yolo config file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
# optimally maxBatches are selected by no_of_classes * 2000
!sed -i 's/max_batches = 500200/max_batches = 7000/' cfg/yolov3_training.cfg
# (classes + 5) x 3 
!sed -i '610 s@classes=80@classes=5@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=5@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=5@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=30@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=30@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=30@' cfg/yolov3_training.cfg

In [8]:
# Make class list file (Person, Bus, Car, Bicycle and TrafficLight)
!echo -e 'person\ncar\nbicycle\nbus\ntraffic light' > data/obj.names # classNames file
# Make data metafile 
!echo -e 'classes= 5\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download initial weights of darknet53 model
!wget https://pjreddie.com/media/files/darknet53.conv.74

# Extract Training Images

The images need to be inside a zip named "images.zip" and they need to be inside the folder "yolov3" on Google Drive otherwise change the filepath accordingly to suit your needs.

In [ ]:
# Extracting images from images.zip for training 
!unzip /mydrive/yolov3/images.zip -d data/obj

In [ ]:
# Assigning class labels appropriately for all class objects detected in images from their annotation files
import glob
import os
import re

# Image annotations 
txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):

    # Writing in the same path as the images
    # print('FILE_PATH = ',file_path)
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            # print(line)
            numbers = re.findall("[0-9.]+", line)
            # print(numbers)
            if numbers:

                # text = (ClassName, x, y, w, h) for all bounding boxes of all classes
                text = "{} {} {} {} {}".format(numbers[0], numbers[1], numbers[2], numbers[3], numbers[4])
                # print('TEXT = ',text)
                text_converted.append(text)
                # print(i, file_path)

        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

In [12]:
# List all training images
import glob
images_list = glob.glob("data/obj/*.jpg")
# print(images_list)

In [13]:
# Create train file from images_List 
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

# Starting training

In [ ]:
# Start the training
# Trained model will be saved in mydrive/yolov3/

# Without logging 
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

# With logging  
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show > LOG.txt